In [1]:
# !pip install psycopg2
# !pip install google-cloud-sql

In [2]:
import pandas as pd
import numpy as np
import pickle
import json
import re

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import psycopg2

In [3]:
# #@markdown Please fill in the value below with your GCP project ID and then run the cell.

# # Please fill in these values.
# project_id = "digital-display-376201" #@param {type:"string"}

# # Quick input validations.
# assert project_id, "⚠️ Please provide a Google Cloud project ID"

# # Configure gcloud.
# !gcloud config set project {project_id}

In [4]:
# from google.cloud.sql.connector import Connector
# import sqlalchemy

In [5]:
# preparamos o driver do Chrome
driver = webdriver.Chrome('path/chromedriver')

<ipython-input-5-c16ba3ec765d>:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('path/chromedriver')


In [6]:
# !dir

In [7]:
# abrir o arquivo de entrada
file = 'parte00.csv'
# file = 'data_output/ztrending_movies.csv'

df0 = pd.read_csv(file, header=None).head(30)
df0 = df0.join(df0[1].apply(json.loads).apply(pd.Series))
df0 = df0.loc[:,['id','original_title']]

In [8]:
# df0.id

# Criacao do dataset com os dados do Rotten Tomatoes

In [9]:
# criamos esta funcao para que os nomes dos filmes sejam transformados nos nomes dos filmes da url
# exemplo: The Matrix ---> the_matrix, desta forma conseguimos acessar as urls

def rottenize(title):
    '''
    Esta funcao sustitui os carateres por vazio ou por _
    para depois entrar nos links (urls) dos filmes no rotten
    '''
    title = title.translate(str.maketrans({':': '', "'": '', ".":"", ",":"", 
                                          "-":"_", " ":"_"}))
    
    return title.lower()

In [14]:
# criamos uma lista vazia para armazenar as urls dos filmes do dataset do TMDB
links = []
for tit in df0['original_title'].apply(rottenize):
    links.append('https://www.rottentomatoes.com/m/{}'.format(tit))

In [15]:
def create_df(n):
    '''
    n: numero de filmes a coletar
    '''  
    # criamos uma lista vazia para armazenar as urls dos filmes do dataset do TMDB
    links = []
    for tit in df0['original_title'].apply(rottenize):
        links.append('https://www.rottentomatoes.com/m/{}'.format(tit))
    
    # listas para armazenar a informacao
    tmdb_id = []

    nomes = []
    TOMATOMETER = []
    TOMATOMETER_rev = []
    aud_scr = []
    aud_scr_rev = []
    main6 = []
    main7 = []
    main8 = []
    main9 = []

    genre = []
    language = []
    director = []
    producer = []
    writer = []
    rel_date_th = []
    rel_date_stream = []
    box = []
    runtime = []
    distributor = []
    description = []
    
    # neste dataset iremos armazenar as informacoes extraidas do Rotten Tomatoes
    df = pd.DataFrame()

    for j in range(len(links[:n])):
        
        # mostra quantos filmes ja varreu
        print(j, end="\r", flush=True)
        
        # entramos no site do filme
        driver.get(links[j])
        
        # se aparecer algun erro no navegador voltamos ao comeco do loop
        try:
            # informacoes da descricao do filme
            info_we = driver.find_elements(By.CLASS_NAME, "media-body")
            # informações mais breves da descricao
            short_info = info_we[0].text.split('\n')[2:]

            # informacoes principais do filme: nome, scores e quantidade de reviews
            info_main = driver.find_elements(By.CLASS_NAME, "thumbnail-scoreboard-wrap")
            info_main = info_main[0].text.split('\n')

            # dicionario para armazenar as infos do filme
            short_info_dic = dict()

            # acrescentamos tambem as informacoes principais
            short_info_dic['Nome'] = info_main[0]
            short_info_dic['TOMATOMETER'] = info_main[2]
            short_info_dic['TOMATOMETER reviews'] = info_main[4]
            short_info_dic['AUDIENCE SCORE'] = info_main[5]
            short_info_dic['AUDIENCE SCORE reviews'] = info_main[7]
            
            # colunas auxiliares
            short_info_dic['main6'] = info_main[1]
            short_info_dic['main7'] = info_main[3]
            short_info_dic['main8'] = info_main[6]
            
            try:
                short_info_dic['main9'] = info_main[8]
            except:
                continue
            
        except:
            continue

        # informacoes breves do filme
        for i in range(len(short_info)):
            infos = short_info[i].split(': ')
            try:
                short_info_dic[infos[0]] = infos[1]
            except:
                short_info_dic[infos[0]] = np.nan

        # resumo e descricao do filme
        short_info_dic[info_we[0].text.split('\n')[0]] = info_we[0].text.split('\n')[1]

        # colocamos as informacoes nas listas
        try:
            nomes.append(short_info_dic['Nome'])
            tmdb_id.append(df0.loc[[j],['id']].values[0][0])
        except:
            nomes.append(np.nan)
        
        try:
            TOMATOMETER.append(short_info_dic['TOMATOMETER'])
        except:
            TOMATOMETER.append(np.nan)
        
        try:
            TOMATOMETER_rev.append(short_info_dic['TOMATOMETER reviews'])
        except:
            TOMATOMETER_rev.append(np.nan)
        
        try:
            aud_scr.append(short_info_dic['AUDIENCE SCORE'])
        except:
            aud_scr.append(np.nan)
        
        try:
            main6.append(short_info_dic['main6'])
        except:
            main6.append(np.nan)
            
        try:
            main7.append(short_info_dic['main7'])
        except:
            main7.append(np.nan)
            
        try:
            main8.append(short_info_dic['main8'])
        except:
            main8.append(np.nan)
            
        try:
            main9.append(short_info_dic['main9'])
        except:
            main9.append(np.nan)
        
        try:
            aud_scr_rev.append(short_info_dic['AUDIENCE SCORE reviews'])
        except:
            aud_scr_rev.append(np.nan)
        
        try:
            genre.append(short_info_dic['Genre'])
        except:
            genre.append(np.nan)
        
        try:
            language.append(short_info_dic['Original Language'])
        except:
            language.append(np.nan)
        
        try:
            director.append(short_info_dic['Director'])
        except:
            director.append(np.nan)
        
        try:
            producer.append(short_info_dic['Producer'])
        except:
            producer.append(np.nan)
        
        try:
            writer.append(short_info_dic['Writer'])
        except:
            writer.append(np.nan)
        
        try:
            rel_date_th.append(short_info_dic['Release Date (Theaters)'])
        except:
            rel_date_th.append(np.nan)
        
        try:
            rel_date_stream.append(short_info_dic['Release Date (Streaming)'])
        except:
            rel_date_stream.append(np.nan)
        
        try:
            box.append(short_info_dic['Box Office (Gross USA)'])
        except:
            box.append(np.nan)
        
        try:
            runtime.append(short_info_dic['Runtime'])
        except:
            runtime.append(np.nan)
        
        try:
            distributor.append(short_info_dic['Distributor'])
        except:
            distributor.append(np.nan)
            
        try:
            description.append(short_info_dic['MOVIE INFO'])
        except:
            description.append(np.nan)
            
    # Alimentamos o database

    df = pd.DataFrame() 

    df['tmdb_id'] = tmdb_id
#     df['Movie Name'] = nomes
    df['tomatometer_reviews'] = aud_scr
#     df['AUDIENCE SCORE reviews'] = aud_scr_rev
    # mudou a estrutura do site, entao algumas listas das q foram criadas inicialmente
    # nao coincidem com o nome da coluna
    df['genre_year'] = TOMATOMETER
#     df['TOMATOMETER reviews'] = nomes

    df['movie_name'] = main6
    df['tomatometer_score'] = main7
    df['audience_score'] = main8
    df['audience_reviews'] = main9
    
    df['Genre'] = genre
    df['Original Language'] = language
    df['Director'] = director
    df['Producer'] = producer
    df['Writer'] = writer
    df['Release Date (Theaters)'] = rel_date_th
    df['Release Date (Streaming)'] = rel_date_stream
    df['box office'] = box
    df['Runtime'] = runtime
    df['Distributor'] = distributor
    df['MOVIE INFO'] = description
        
    return df

In [17]:
# df

In [19]:
# %%timeit
df = create_df(len(links))
# df = create_df(4)

In [20]:
df

,tmdb_id,tomatometer_reviews,genre_year,movie_name,tomatometer_score,audience_score,audience_reviews,Genre,Original Language,Director,Producer,Writer,Release Date (Theaters),Release Date (Streaming),box office,Runtime,Distributor,MOVIE INFO
0,5,45 Reviews,"R 1995, Drama, 1h 42m",FOUR ROOMS,13%,69%,"50,000+ Ratings",Drama,English,"Allison Anders, Alexandre Rockwell, Robert Rod...",Lawrence Bender,"Allison Anders, Quentin Tarantino, Robert Rodr...","Dec 25, 1995 Original","Jun 30, 2008",$4.1M,1h 42m,Miramax Films,Working New Year's Eve at a hotel in Hollywood...
1,11,138 Reviews,"PG 1977, Sci-fi/Adventure, 2h 1m",STAR WARS: EPISODE IV -- A NEW HOPE,93%,96%,"250,000+ Ratings","Sci-Fi, Adventure, Action, Fantasy",English,George Lucas,Gary Kurtz,George Lucas,"May 25, 1977 Wide","Dec 6, 2005",$345,2h 1m,NaN,The Imperial Forces -- under orders from cruel...
2,12,269 Reviews,"G 2003, Kids & family/Comedy, 1h 40m",FINDING NEMO,99%,86%,"250,000+ Ratings","Kids & Family, Comedy, Adventure, Animation",English,Andrew Stanton,Graham Walters,"Andrew Stanton, Andrew Stanton, Bob Peterson, ...","May 30, 2003 Wide","Nov 4, 2003",NaN,1h 40m,Walt Disney Pictures,"Marlin (Albert Brooks), a clown fish, is overl..."
3,13,106 Reviews,"PG-13 1994, Comedy/Drama, 2h 22m",FORREST GUMP,71%,95%,"250,000+ Ratings","Comedy, Drama",English,Robert Zemeckis,"Wendy Finerman, Steve Starkey, Steve Tisch","Winston Groom, Eric Roth","Jul 6, 1994 Wide","Aug 28, 2001",$330.3M,2h 22m,Paramount Pictures,Slow-witted Forrest Gump (Tom Hanks) has never...
4,14,191 Reviews,"R 1999, Comedy/Drama, 2h 2m",AMERICAN BEAUTY,87%,93%,"250,000+ Ratings","Comedy, Drama",English,Sam Mendes,"Bruce Cohen, Dan Jinks",Alan Ball,"Oct 1, 1999 Wide","Jan 2, 2002",$130.1M,2h 2m,DreamWorks SKG,Lester Burnham (Kevin Spacey) is a gainfully e...
5,15,127 Reviews,"PG 1941, Drama, 1h 59m",CITIZEN KANE,99%,90%,"100,000+ Ratings",Drama,English,Orson Welles,Orson Welles,"Herman J. Mankiewicz, Orson Welles, John Houseman","May 1, 1941 Wide","Feb 23, 2010",$977.3K,1h 59m,RKO Radio Pictures,When a reporter is assigned to decipher newspa...
6,18,68 Reviews,"PG-13 1997, Sci-fi/Adventure, 2h 7m",THE FIFTH ELEMENT,71%,86%,"250,000+ Ratings","Sci-Fi, Adventure, Action, Fantasy",English,Luc Besson,Patrice Ledoux,"Luc Besson, Robert Mark Kamen","May 9, 1997 Wide","Oct 9, 2001",$63.7M,2h 7m,Columbia Pictures,"In the 23rd century, a New York City cabbie, K..."
7,24,238 Reviews,"R 2003, Action, 1h 50m",KILL BILL: VOL. 1,85%,81%,"250,000+ Ratings",Action,English,Quentin Tarantino,Lawrence Bender,Quentin Tarantino,"Oct 10, 2003 Wide","Apr 13, 2004",$70.1M,1h 50m,Miramax Films,"A former assassin, known simply as The Bride (..."
8,38,253 Reviews,"R 2004, Romance/Sci-fi, 1h 48m",ETERNAL SUNSHINE OF THE SPOTLESS MIND,92%,94%,"250,000+ Ratings","Romance, Sci-Fi",English,Michel Gondry,"Anthony Bregman, Steve Golin","Charlie Kaufman, Michel Gondry","Mar 19, 2004 Original","Sep 28, 2004",$34.1M,1h 48m,Focus Features,"After a painful breakup, Clementine (Kate Wins..."
